In [82]:
import os
import torch
import torchaudio

def preprocess_audio(audio_path):
    # Load the audio file
    waveform, sr = torchaudio.load(audio_path)

    # Print original sample rate
    print(f'Original sample rate: {sr}')

    # Resample to 16kHz
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        waveform = resampler(waveform)
        sr = 16000

    # Print resampled sample rate
    print(f'Resampled sample rate: {sr}')

    # Define the normalization and feature extraction transformation
    transform = torchaudio.transforms.MFCC(sample_rate=sr, n_mfcc=40)

    # Normalize and extract features from the audio
    mfcc = transform(waveform)

    # Prepare the tensor for fine-tuning
    tensor = mfcc.squeeze(0)
    tensor = tensor.transpose(0, 1)

    return tensor

path_audio = '/home/vit-ap/Desktop/Navana AI Intern/AudioRecordings'

for filename in os.listdir(path_audio):
    if filename.endswith('.wav'):
        full_path = os.path.join(path_audio, filename)
        print(f'Processing {filename}')
        tensor = preprocess_audio(full_path)


Processing negative_3.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_4.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_2.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_4.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_1.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_2.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_5.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_3.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_1.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_5.wav
Original sample rate: 44100
Resampled sample rate: 16000


In [83]:
path_audio = '/home/vit-ap/Desktop/Navana AI Intern/AudioRecordings'
path_save = '/home/vit-ap/Desktop/Navana AI Intern/processedFiles'

for filename in os.listdir(path_audio):
    if filename.endswith('.wav'):
        full_path = os.path.join(path_audio, filename)
        print(f'Processing {filename}')
        tensor = preprocess_audio(full_path)
        save_path = os.path.join(path_save, f'{filename.split(".")[0]}.pt')
        torch.save(tensor, save_path)


Processing negative_3.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_4.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_2.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_4.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_1.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_2.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_5.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_3.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing negative_1.wav
Original sample rate: 44100
Resampled sample rate: 16000
Processing positive_5.wav
Original sample rate: 44100
Resampled sample rate: 16000


In [84]:
class AudioDataset(Dataset):
    def __init__(self, path_audio, path_save):
        self.path_audio = path_audio
        self.path_save = path_save
        self.files = os.listdir(path_save)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        tensor = torch.load(os.path.join(self.path_save, file))
        label = int(file.split('_')[0]) # Assumes file name format is 'label_filename.pt'
        return tensor, label

path_audio = '/home/vit-ap/Desktop/Navana AI Intern/AudioRecordings'
path_save = '/home/vit-ap/Desktop/Navana AI Intern/processedFiles'
dataset = AudioDataset(path_audio, path_save)

In [85]:
# Import necessary modules
import torch
from torch.utils.data import Dataset
from transformers import AutoModelWithLMHead, AutoTokenizer

# Define the model and tokenizer
model = AutoModelWithLMHead.from_pretrained("facebook/wav2vec2-base")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base")

# Set the model to fine-tune mode
model.train()

# Define the path to your audio files and processed tensors
path_audio = '/home/vit-ap/Desktop/Navana AI Intern/AudioRecordings'
path_save = '/home/vit-ap/Desktop/Navana AI Intern/processedFiles'


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForMaskedLM: ['project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForMaskedLM were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to

In [86]:
# Define your dataset
from torch.utils.data import Dataset

label_dict = {'positive': 1, 'negative': 0}
class AudioDataset(Dataset):
    def __init__(self, path_audio, path_save):
        self.path_audio = path_audio
        self.path_save = path_save
        self.files = [f for f in os.listdir(path_save) if f.endswith('.pt')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        tensor = torch.load(os.path.join(self.path_save, file))
        label = label_dict[file.split('_')[0]]
        print(f'Tensor size: {tensor.size()}')
        return tensor, label
    

In [87]:

# Define your dataset
from torch.utils.data import Dataset

label_dict = {'positive': 1, 'negative': 0}
class AudioDataLoader(torch.utils.data.Dataset):
    def __init__(self, path_audio, path_save):
        self.path_audio = path_audio
        self.path_save = path_save
        self.files = [f for f in os.listdir(path_save) if f.endswith('.pt')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        tensor = torch.load(os.path.join(self.path_save, file))
        label = label_dict[file.split('_')[0]]
        print(f'Tensor size: {tensor.size()}') # Assumes file name format is 'label_filename.pt'
        return tensor, label
    
    def collate_fn(batch):
        inputs = torch.nn.utils.rnn.pad_sequence([b[0] for b in batch], batch_first=True)
        labels = torch.tensor([b[1] for b in batch])
        return inputs, labels

In [88]:
print(f'Tensor size: {tensor.size()}')

Tensor size: torch.Size([40, 2, 1769])


In [89]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=40, shuffle=True, collate_fn=collate_fn)

In [90]:


# Define your optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the model
for epoch in range(10):
    for i, (inputs, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


RuntimeError: [enforce fail at inline_container.cc:129] . file in archive is not in a subdirectory: mimetype

RuntimeError: stack expects each tensor to be equal size, but got [40, 2, 1769] at entry 0 and [40, 2, 1093] at entry 1